In [2]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
from steam import WebAPI
import os
import glob
import tensorflow as tf
import requests
from sklearn import decomposition

#import logging
#logging.getLogger("tensorflow").setLevel(logging.ERROR)

#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

#dont keep api key in github!!!
api_file = os.path.join('..','..','..','apikeys','steam_api_key.txt')
with open(api_file, 'r') as fin:
    api = WebAPI(key=fin.readline())

print(tf.__version__)

2.0.0


In [ ]:
sns.set_style('ticks')

In [ ]:
#declare interface urls
#dota2_beta should be used for testing in order to not aggressively make API requests
dota2_id = '570'
dota2_beta_id = '205790'

hero_list = api.call('IEconDOTA2_'+dota2_id+'.GetHeroes')
hero_vocab=[int(hero_id['id']) for hero_id in hero_list['result']['heroes']] 
print(len(hero_vocab))

# Load embedding from tmp_log_dir or log_dir

In [ ]:
#if you know the log_dir of a specific embedding
logdir = os.path.join('..','..','..',
                      'tmp_log_dir',
                      '20191029-164616',
                      'embedding.npy')
#logdir = os.path.join('log_dir',
#                        '20191029-164616',
#                        'embedding.npy')
embedding = log_dir

In [8]:
#if you don't know the log_dir of embedding or
#  you want to get multiple embeddings
paths=glob.glob(os.path.join('..','..','..',
                             'tmp_log_dir','*'))
#files=glob.glob(os.path.join('log_dir'))
paths.sort(key=os.path.getmtime)

##load most recent embedding
#embedding = np.load(os.path.join(paths[0],
#                                 'embedding.npy'))

#load all embeddings in logdir
embedding_dict = {}
for path in paths:
    #15 characters are in the datetime stamp
    #  of the log folders
    embedding_dict[path[-15:]]=np.load(
                            os.path.join(
                            path,'embedding.npy'))

20191029-164616
{'20191029-164616': array([[-3.56099606e-02,  5.82716614e-03,  2.97715701e-02],
       [ 4.56626080e-02,  3.22495140e-02, -5.87306917e-04],
       [ 4.36755680e-02, -1.44680142e-02,  1.33134611e-02],
       [-1.25045665e-02, -2.21487768e-02, -7.06342608e-03],
       [-1.48785114e-03, -4.37726155e-02,  8.93944502e-03],
       [-2.04344746e-02,  3.24770696e-02,  1.57090761e-02],
       [ 1.79680102e-02, -3.87679338e-02,  4.10391353e-02],
       [-2.99699064e-02,  3.14247124e-02,  8.12405348e-03],
       [-3.85280475e-02,  4.64451574e-02,  1.21403225e-02],
       [ 4.84262444e-02, -3.51472721e-02,  1.44391768e-02],
       [ 2.49458589e-02,  1.56010725e-02,  3.13552953e-02],
       [-2.48796865e-03, -3.71026173e-02, -7.12900236e-03],
       [-4.90677133e-02, -2.85258181e-02, -2.45053172e-02],
       [ 1.04617588e-02, -2.06429716e-02, -1.53941885e-02],
       [ 3.11994441e-02, -4.87801544e-02, -3.81054766e-02],
       [ 3.01805884e-03,  3.84063236e-02, -1.86521523e-02],
    

In [ ]:
for key,value in embedding_dict:
    

In [ ]:
pca = decomposition.PCA(n_components=2)
pca_embedding = pca.fit_transform(embedding)

In [ ]:
for vec,hero in zip(pca_embedding,hero_list['result']['heroes']):
    plt.text(vec[0],vec[1],hero['name'][14:])
    
plot_range = max(embedding[:,0])
plt.xlim(-plot_range*1.15,plot_range*1.3)
plt.ylim(-plot_range*1.15,plot_range*1.3)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(30,30))

plot_range = 1.3*max(embedding[:,0])

for vec,hero in zip(embedding,hero_list['result']['heroes']):
    if hero['name'][14:]=='techies':
        font_dict = {'color': 'red',
                     'size': 8}
    else:
        font_dict = {'color': 'black',
                     'size': 8}

    ax[0,0].text(vec[1],vec[0],hero['name'][14:],fontdict=font_dict)
    ax[0,1].text(vec[2],vec[0],hero['name'][14:],fontdict=font_dict)
    ax[1,1].text(vec[1],vec[2],hero['name'][14:],fontdict=font_dict)
    ax[1,0].axis('off')
# Do a little setup to make sure the plot displays nicely.
for i in [0,1]:
    for j in [0,1]:
        ax[i,j].set_xlim([-plot_range,plot_range])
        ax[i,j].set_ylim([-plot_range,plot_range])
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])
        
plt.subplots_adjust(wspace=0, hspace=0)
plt.show() 